In [1]:
import cv2 as cv
import numpy as np

In [4]:
trackCornerParams = dict(maxCorners = 20, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

# greater winSize =  smaller movements hard to detect
# smaller winSize =  too much noise means too many movements are detected
# set default = 200,200

# maxLevel is some kind of Pyramid level
# TERM_CRITERIA_COUNT = 10 here (defines the number of iterations) ( more iterations = better results but takes time)
# TERM_CRITERIA_EPS = 0.03 (epslon) (defines the time for which the iterations run) ( more time = better results)
# among the two if one exhausts the iterations stop
lk_params = dict(winSize = (50, 50), maxLevel = 5, criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 20, 0.1))

cap = cv.VideoCapture(0)

ret, prevFrame = cap.read()

grayPrev = cv.cvtColor(prevFrame, cv.COLOR_BGR2GRAY)

# i should call it previous points as these are the points of the previous frame which are tracked
# instead of trackPoints
prevPts = cv.goodFeaturesToTrack(grayPrev, mask = None, **trackCornerParams)

mask = np.zeros_like(prevFrame)

while True:
    
    ret, frame = cap.read()
    
    grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Pyramid Lucas Kanade method
    
    # nextPts =  output vector of 2D points
    # status = vector of unsigned chars with value either 0 or 1(if flow for corresponding element has been found)
    # err = for errors
    nextPts, status, err = cv.calcOpticalFlowPyrLK(grayPrev, grayFrame, prevPts, nextPts = None, **lk_params)
    
    # these two contain old and new coordinates of tracked points
    goodElementsNew = nextPts[status == 1]
    goodElementsOld = prevPts[status == 1]
    
    for i, (new, old) in enumerate(zip(goodElementsNew, goodElementsOld)):
        
        xNew, yNew = new.ravel()
        xOld, yOld = old.ravel()
        
        mask = cv.line(mask, (xNew, yNew), (xOld, yOld), (0, 255, 0), 3)
        
        frame = cv.circle(frame, (xNew, yNew), 5, (0, 0, 255), -1)
        
    img = cv.add(mask, frame)
        
    cv.imshow('Tracking', img)
        
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
    grayPrev = grayFrame.copy()
    prevPts = goodElementsNew.reshape(-1, 1, 2)

        
cv.destroyAllWindows()
cap.release()